In [1]:
addprocs()
using SortSpikes, ExtractSpikes, DistributedArrays

In [4]:
#Parallel Processing template

st=Array(Sorting,64)
for i=1:64
    st[i]=Sorting(SpikeDetection(),Cluster(),ones(Int64,20000),zeros(Int,500),zeros(Int,500),2)
    st[i].rawSignal[200:700:end]=10*st[i].rawSignal[200:700:end]
end

s=distribute(st);

@time map!(onlineCal, s);
@time map!(onlineSort, s);

  74.919 milliseconds (3057 allocations: 244 KB)
  11.734 milliseconds (3063 allocations: 244 KB)


In [6]:
#pyqtgraph testing

using PySide
reload(Pkg.dir("PySide", "src", "pyqtgraph.jl"))
using PyQtGraph
using PyCall
w = Widget()
lyt = VBoxLayout(w)
setLayout(w, lyt)
## Graphics Layout Widget make grid of plot devices
win = GraphicsLayoutWidget(w)
addWidget(lyt, win)
set_size(w, 800, 600)
raise(w)
## addPlot adds a plot device in next column over and returns a plot object
p1 = addPlot(win, title="Basic array plotting")
p1.plot(randn(100))
p2 = addPlot(win, title="Multiple curves")
p2.plot(randn(100), pen=(255,0,0))
p2.plot(randn(100) .+ 5, pen=(0, 255, 0))
p2.plot(randn(100) .+ 10, pen=(0,0,255))
p3 = addPlot(win, title="Drawing with points")
p3.plot(randn(100), pen=(200, 200, 200), symbolBrush=(255,0,0), symbolPen="w")
nextRow(win)
p4 = addPlot(win, title="Parametric, grid enabled")
p4.plot(sin(linspace(0, 2pi, 1000)), cos(linspace(0, 2pi, 1000)))
p4.showGrid() # defaults x=true, y=true
p5 = addPlot(win, title="Scatter plot, axis labels, log scale")
x = 1e-5 * rand(1000)
y = x*1000 .+ 0.005 * randn(1000)
y -= minimum(y).-1.0
mask = x .> 1e-15
x = x[mask]
y = y[mask]
p5.plot(x, y, pen=nothing, symbol="t", symbolPen=nothing, symbolSize=10, symbolBrush=(100, 100, 255, 50))
p5.setLabel("left", text="Y Axis", units="A")
p5.setLabel("bottom", text="Y Axis", units="s")
p5.setLogMode(x=true, y=false)
p6 = addPlot(win, title="Updating plot")
curve = p6.plot(pen="y")
data = randn(10000, 10)
ptr = 0
function update()
global curve, data, ptr, p6
qinvoke(curve, :setData, 1:10_000, data[:, 1 + ptr]) # had to fix this
if ptr == 0
p6.enableAutoRange("xy", false) ## stop auto-scaling after the first data set is plotted
end
ptr = (ptr + 1) % 10
end
timer = QtCore[:QTimer]()
qconnect(timer, :timeout, update)
qinvoke(timer, :start, 50)
nextRow(win)
p7 = addPlot(win, title="Filled plot, axis disabled")
y = sin(linspace(0.0, 10, 1000)) .+ randn(1000)*0.1
p7.plot(y, fillLevel=-0.3, brush=(50,50,200,100))
p7.showAxis("bottom", show=false)
p8 = addPlot(win, title="Region Selection")
x2 = linspace(-100, 100, 1000)
data2 = sin(x2) ./ x2
p8.plot(data2, pen=(255,255,255,200))
## no special functions for LinearRegionItem
lr = pyqtgraph.(:LinearRegionItem)([400,700])
qinvoke(lr, :setZValue, -10)
p8.addItem(lr)
p9 = addPlot(win, title="zoom")
p9.plot(data2)
function updatePlot()
p9.setXRange(lr[:getRegion]()..., padding=0)
end
function updateRegion()
view_range = qinvoke(p9, [:getViewBox, :viewRange]) # 2x2 array
x_range = Int[view_range[1,j] for j in 1:2] # a vector, not just view_range[1,:]'
lr[:setRegion](x_range)
end
qconnect(lr, :sigRegionChanged, updatePlot)
#qconnect(p9, :sigXRangeChanged, updateRegion) ## this is giving issues. Find out why.
updatePlot()
#raise(w)

 in depwarn at ./deprecated.jl:62
 in ntuple at deprecated.jl:49
 in convert at /home/paul/.julia/v0.4/PyCall/src/conversions.jl:222
 in call at /home/paul/.julia/v0.4/PyCall/src/numpy.jl:218
 in convert at /home/paul/.julia/v0.4/PyCall/src/numpy.jl:427
 in convert at /home/paul/.julia/v0.4/PyCall/src/conversions.jl:846
 in pywrap at /home/paul/.julia/v0.4/PyCall/src/PyCall.jl:311
 in pywrap at /home/paul/.julia/v0.4/PyCall/src/PyCall.jl:304
 in include at ./boot.jl:253
 in include_from_node1 at ./loading.jl:133
 in reload_path at ./loading.jl:157
 in reload at loading.jl:87
 in include_string at loading.jl:99
 in execute_request_0x535c5df2 at /home/paul/.julia/v0.4/IJulia/src/execute_request.jl:157
 in eventloop at /home/paul/.julia/v0.4/IJulia/src/IJulia.jl:123
 in anonymous at task.jl:365
while loading /home/paul/.julia/v0.4/PySide/src/pyqtgraph.jl, in expression starting on line 363
 in depwarn at ./deprecated.jl:62
 in ntuple at deprecated.jl:49
 in convert at /home/paul/.julia/v0

In [47]:
#GUI Experiments

# plot in an external window since it doesn't work yet in IJulia
pygui(true)
# tell PyPlot that the plot is interactive
PyPlot.ion()
# . . . and that previous plots are overwritten
#PyPlot.hold(false)

(fig, ax) = plt.subplots(8,8)
x=[1:1:100]
y=[rand(100)]
li=[hi[:plot](x, y)[1] for hi in ax]

fig[:canvas][:draw]()
plt.show()

y2=[rand(500,100)]

@time for i=1:10
    
    #[fig[:canvas][:restore_region](hi) for hi in backgrounds]
    [hi[:set_ydata](y2[i,:]) for hi in li]
    [fig[:canvas][:blit](hi[:bbox]) for hi in ax]
    #fig[:canvas][:draw]()
end

elapsed time: 1.072894188 seconds (471040 bytes allocated)


In [38]:
#Wavelet Experiments

function cwt(x::Array{Float64,1},t::Int,psi::Array{Float64,1},bs::Array{Float64,1})

    bs[1]=bs[2]-x[t-20]*psi[1]+x[t]*psi[21]

    for i=2:19
        bs[i]=bs[i+1]-x[t-20]*psi[i]+x[t]*psi[22-i] 
    end

    bs[20]=0.0
    for i=1:20
        bs[20]+=x[t-20+i]*psi[21-i]
    end
    
    return bs
end


function mycwt(x::Array{Float64,1},t::Int, psi::Array{Float64,1})
    
    bs=0.0
    
    for i=1:20
        bs+=x[t-20+i]*psi[21-i]
    end
    
end

function main()
    for i=1:20000
        mycwt(x,t,psi)
    end
end

psi=[0.00021208083980379827
0.00035858968789573785
-0.0021782363581090178
-0.004159358781386048
0.010131117519849788
0.023408156785839195
-0.02816802897093635
-0.09192001055969624
0.05204316317624377
0.4215662066908515
-0.7742896036529562
0.4379916261718371
0.06203596396290357
-0.10557420870333893
-0.0412892087501817
0.03268357426711183
0.01976177894257264
-0.009164231162481846
-0.006764185448053083
0.0024333732126576722
0.0016628637020130838
-0.0006381313430451114
-0.00030225958181306315
0.00014054114970203437
4.134043227251251e-05
-2.1315026809955787e-05
-3.7346551751414047e-06
2.0637618513646814e-06
1.6744288576823017e-07
-9.517657273819165e-08
];

bs=zeros(Float64,20)
    
x=rand(100)
t=50

for j=1:20
    for i=1:20
        bs[j]+=x[t-20+i]*psi[21-i]
    end
end

main (generic function with 1 method)